# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])

import re
import time
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import LinearSVC
import xgboost as xgb

[nltk_data] Downloading package punkt to
[nltk_data]     /home/derricklewis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/derricklewis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [5]:
def tokenize(text):
        # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [54]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
start = time.time()
pipeline.fit(X_train, y_train)
print( time.time()-start)

80.011390209198


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
def report_results(test, pred):
    results_df=pd.DataFrame(columns=['precision', 'recall', 'f1-score', 'support', 'accuracy'])
    for index, column in enumerate(test.columns):
        cr_dict = classification_report(test[column], pred[:,index], output_dict=True, labels=np.unique(pred[:,index]))
        cr_dict['weighted avg']['accuracy'] = accuracy_score(test[column], pred[:,index])
        results_df = results_df.append(pd.DataFrame(index=[column], data=cr_dict['weighted avg']))
    return results_df

In [10]:
y_pred = pipeline.predict(X_test)

In [16]:
first_trial = report_results(y_test, y_pred)
first_trial.mean()

precision       0.937981
recall          0.946886
f1-score        0.932576
support      6545.388889
accuracy        0.945572
dtype: float64

### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [17]:
start = time.time()
parameters = {
    # 'vect__max_df': (0.5, 0.75, 1.0),
    # 'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200],
    # 'clf__estimator__min_samples_split': [2, 3],
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)
cv.fit(X_train, y_train)
print(time.time()-start)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END ................clf__estimator__n_estimators=50; total time=  34.3s
[CV 2/5] END ................clf__estimator__n_estimators=50; total time=  33.1s
[CV 3/5] END ................clf__estimator__n_estimators=50; total time=  34.3s
[CV 4/5] END ................clf__estimator__n_estimators=50; total time=  33.6s
[CV 5/5] END ................clf__estimator__n_estimators=50; total time=  33.6s
[CV 1/5] END ...............clf__estimator__n_estimators=100; total time=  52.5s
[CV 2/5] END ...............clf__estimator__n_estimators=100; total time=  52.6s
[CV 3/5] END ...............clf__estimator__n_estimators=100; total time=  55.7s
[CV 4/5] END ...............clf__estimator__n_estimators=100; total time=  59.8s
[CV 5/5] END ...............clf__estimator__n_estimators=100; total time=  59.3s
[CV 1/5] END ...............clf__estimator__n_estimators=200; total time= 1.6min
[CV 2/5] END ...............clf__estimator__n_est

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
y_pred = cv.predict(X_test)

In [20]:
gscv_trial = report_results(y_test, y_pred)
gscv_trial.mean()

precision       0.939004
recall          0.946661
f1-score        0.932356
support      6547.416667
accuracy        0.945657
dtype: float64

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [7]:
pipeline_2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LinearSVC()))
])
pipeline_2.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__C', 'clf__estimator__class_weight', 'clf__estimator__dual', 'clf__estimator__fit_intercept', 'clf__estimator__intercept_scaling', 'clf__estimator__loss', 'clf__estimator__max_iter', 'clf__estimator__multi_class', 'clf__estimator__penalty', 'clf__estimator__random_state', 'clf__estimator__tol', 'clf__estimator__verbose', 'clf__estimator', 'clf__n_jobs'])

In [11]:
start = time.time()
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'tfidf__use_idf': (True, False),
    'clf__estimator__C': [.5, .75, 1.0],
    # 'clf__estimator__min_samples_split': [2, 3],
    
}

cv_2 = GridSearchCV(pipeline_2, param_grid=parameters, verbose=3, cv=3)
cv_2.fit(X_train, y_train)
print(time.time()-start)

clf__estimator__C=0.75, tfidf__use_idf=False, vect__max_df=1.0; total time=   6.5s
[CV 2/3] END clf__estimator__C=0.75, tfidf__use_idf=False, vect__max_df=1.0; total time=   6.2s
[CV 3/3] END clf__estimator__C=0.75, tfidf__use_idf=False, vect__max_df=1.0; total time=   6.1s
[CV 1/3] END clf__estimator__C=1.0, tfidf__use_idf=True, vect__max_df=0.5; total time=   6.1s
[CV 2/3] END clf__estimator__C=1.0, tfidf__use_idf=True, vect__max_df=0.5; total time=   6.0s
[CV 3/3] END clf__estimator__C=1.0, tfidf__use_idf=True, vect__max_df=0.5; total time=   6.0s
[CV 1/3] END clf__estimator__C=1.0, tfidf__use_idf=True, vect__max_df=0.75; total time=   6.1s
[CV 2/3] END clf__estimator__C=1.0, tfidf__use_idf=True, vect__max_df=0.75; total time=   6.2s
[CV 3/3] END clf__estimator__C=1.0, tfidf__use_idf=True, vect__max_df=0.75; total time=   6.0s
[CV 1/3] END clf__estimator__C=1.0, tfidf__use_idf=True, vect__max_df=1.0; total time=   5.9s
[CV 2/3] END clf__estimator__C=1.0, tfidf__use_idf=True, vect__m

NameError: name 'report_results' is not defined

In [13]:
y_pred = cv_2.predict(X_test)
linearSVC_trial = report_results(y_test, y_pred)
linearSVC_trial.mean()

precision       0.942676
recall          0.950935
f1-score        0.943358
support      6547.828571
accuracy        0.949993
dtype: float64

## 8.5. XGboost
- testing XGboost

In [14]:
pipeline_3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss', 
    use_label_encoder=False)))
])
pipeline_3.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__objective', 'clf__estimator__use_label_encoder', 'clf__estimator__base_score', 'clf__estimator__booster', 'clf__estimator__colsample_bylevel', 'clf__estimator__colsample_bynode', 'clf__estimator__colsample_bytree', 'clf__estimator__gamma', 'clf__estimator__gpu_id', 'clf__estimator__importance_type', 'clf__estimator__interaction_constraints', 'clf__estimator__learning_rate', 'clf__estimator__max_delta_step', 'clf__estimator__max_depth', 'clf__estimator__min_child_weight', 'clf__estimator__missing', 'clf__estimat

In [15]:

start = time.time()
parameters = {
    # 'vect__max_df': (0.5, 0.75, 1.0),
    # 'tfidf__use_idf': (True, False),
    # 'clf__estimator__gamma': [.5, .75, 0],
    # 'clf__estimator__min_samples_split': [2, 3],
    
}

cv_3 = GridSearchCV(pipeline_3, param_grid=parameters, cv=4, verbose=3, n_jobs=-1)
cv_3.fit(X_train, y_train)
print(time.time()-start)
y_pred = cv_3.predict(X_test)
first_trial = report_results(y_test, y_pred)
first_trial.mean()

Fitting 4 folds for each of 1 candidates, totalling 4 fits


KeyboardInterrupt: 

In [ ]:
cv_2

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.